# Configuration file
The cell below contains the configuration file that will allow the execution of Django code inside Jupyter Notebook

In [ ]:
import os, sys
PWD = os.getenv('PWD')
os.chdir(PWD)
sys.path.insert(0, os.getenv('PWD'))
os.environ.setdefault("DJANGO_SETTINGS_MODULE", "local_settings.py")
import django
django.setup()

## Write your code below

#####  6.4.7 Make Choices and Sub-Choices Model Constants

A nice pattern is to add choices as properties to a model. As these are constants tied to your model (and the represented data) being able to easily access them everywhere makes development easier.
This technique is described in https://docs.djangoproject.com/en/1.11/ref/models/ fields/#choices. If we translate that to an ice cream-based example, we get:
Example 6.5: Setting Choice Model Attributes
 



In [ ]:
from orders.models import IceCreamOrder
IceCreamOrder.objects.filter(flavor=IceCreamOrder.FLAVOR_CHOCOLATE)

In [ ]:
from reviews.models import FlavorReview
FlavorReview.objects.count()

FlavorReview.objects.published().count()

#### 6.7.1 Model Behaviors a.k.a Mixins

Model behaviours embrace the idea of compositionn and encapsulation via the use of mixins. Models inherit logic from abstrac models. For more information, see the following resources:

* blog.kevinstone.com/django-model-behaviors.html -> Kevin Stone's article on using composition to reduce replication of code.

* medium.com/eshares-blog/supercharging-django-productivity-8dbf9042825e -> includes a really good section on using **DateTimeField** for logical deletes.

* Section 10.2: Using Mixins With CBVs.

# Kevin Stone

### Django Model Behaviors

As Django projects scale in complexity beyond the neat and tidy tutorial phase, how can we structure our models to keep things manageable? We're talking 10s 100s og models, used across numerous views, templates and tests...


### Compositional Model Behaviors

The Compositional Model pattern allows you to manage the complexity of your models through compartmentalization of functionality into manegeable components.

#### The Benefits of Fat Models

* Encapsulation
* Single Path
* Seperation of Concerns (MVC)

#### Without the Maintenance Cost

* DRY
* Readability
* Reusability
* Single Responsibility Principle
* Testability

### Model Behaviours Example


In [ ]:
# Traditional model

class BlogPost(models.Model):
    title = models.CharField(max_length=255)
    body = models.TextField()
    slug = models.SlugField()
    author = models.ForeignKey(User, related_name='posts')
    create_date = models.DateTimeField(auto_now=True)
    modified_date = models.DateTimeField(auto_now=True)
    publish_date = models.DateTimeField(null=True)

### Decomposed into Discrete Behaviors

The goal of hte behavior pattern is to decompose your models into core, reusable mixins. Create a higher level abstraction than the model field that encapsulates the intended business logic.

In [ ]:
from .behaviors import (
    Authorable, Permalinkable, Timestampable, Publishable)

class BlogPost(
    Authorable, Permalinkable, Timestampable, Publishable):
    title = models.CharField(max_length=255)
    body = models.TextField()

In [ ]:
# Reusable Behaviors

class Authorable(models.Model):
    author = models.ForeignKey(User)
    
    class Meta:
        abstract = True

        
class Permalinkable(models.Model):
    slug = models.SlugField()
    
    class Meta:
        abstract = True
        

 class Publishable(models.Model):
    publish_date = models.DateTimeField(null=True)
    
    class Meta:
        abstract =True
        
        
class Timestampable(models.Model):
    create_date = models.DateTimeField(auto_now_add=True)
    modified_date = models.DateTimeField(auto_now=True)
    
    class Meta:
        abstract = True

## Models are more than just Fields

Our first cut at common behaviors just captured common fields, but what about everything else models encapsulate?

* Properties
* Custom Methods
* Method Overloads(save(), etc...)
* Validation
* Querysets

#### Capturing Model Methods

Let's extend our traditional fat model with some of these encapsulated business logic

In [ ]:
class BlogPost(models.Model):
    
    @property
    def is_published(self):
        from django.utils import timezone
        return self.publish_date < timezone.now()
    
    @models.permalink
    def get_absolute_url(self):
        return('blog-post', (), {
            "slug": self.slug,
        })
    
    def pre_save(self, instance, add):
        from django.utils.text import slugify
        if not instance.slug:
            instance.slug = slugify(self.title)
            

### Behaviors with Methods

In actuality, these same methods can be generalized and extracted into our behaviors models

In [ ]:
class Permalinkable(models.Model):
    slug = models.SlugField()
    
    class Meta:
        abstract = True
        
    
    def get_url_kwargs(self, **kwargs):
        kwargs.update(getattr(self, 'url_kwargs', {}))
        return kwargs
    
    @models.permalink
    def get_absolute_url(self):
        url_kwargs = self.get_url_kwargs(slug=self.slug)
        return (self.url_name, (), url_kwargs)
    
    def pre_save(self, instance, add):
        from django.utils.text import slugify
        if not instance.slug:
            instance.slug = slugify(self.slug_source)

#### Wire up the Concrete Model

Since we generalized our behaviours, we need to add some helpers on our concrete models to complete the functionality.

In [ ]:
from .behaviours import Authorable, Permalinkable, Timestampable, Publishable

class BlogPost(Authorable, Permalinkable, Timestampable, Publishable, models.Model):
    title = models.CharField(max_length=255)
    body = models.TextField()
    
    url_name = "blog-post"
    
    @property
    def slug_source(self):
        return self.title

## Naming Tips
Use "<verb>-able" naming pattern for behaviors. The "-able" suffix ensures the behaviors are readily identifiable. It also prevents yet another use of the word Mixin. (Don't worry when the naming deviates from decent english such as in the case of OptionallyGenericRelateable)
    
## Custom Queryset Chaining
We all know to chain queryset methods, but what about adding custom manager methods?

Let's Find Posts from a Given Author (username) that are Published(publish__date in the past)

### QuerySet without Encapsulation

In [ ]:
from django.utils import timezone
from .models import BlogPost

BlogPost.objects.filter(author__username='username1')\
.filter(publish_date__lte=timezone.now())

### Custom Managers
Let's create methods on a custom Manager to handle the past publication date and author filters.

In [ ]:
class BlogPostManager(models.Manager):
    
    def published(self):
        from django.utils import timezone
        return self.filter(publish_date__lte=timezone.now())
    
    def authored_by(self, author):
        return self.filter(author__username=author)
    

class BlogPost(models.Model):
    
    objects = BlogPostManager()

In [3]:
import time
hours, rem = divmod(79.12949585914612, 3600)
print(hours)
print(rem)

0.0
79.12949585914612


In [4]:
minutes, seconds = divmod(rem, 60)
print(minutes)
print(seconds)

1.0
19.129495859146118
